In [ ]:
import crystal_toolkit
import os
from pymatgen.core import Structure

In [ ]:
cif_name = "gen_4"

struct_expt = Structure.from_file(
    os.path.join("cif", f"{cif_name}.cif")
)
print(struct_expt)

struct_expt

In [ ]:
# get reduced formula
reduced_formula = struct_expt.composition.reduced_formula

# mkdir in output folder named reduced_formula
os.makedirs(f"output/{reduced_formula}", exist_ok=True)

## pbe

In [ ]:
# Relax with pbe

import matcalc

calculator_pbe = matcalc.load_fp("pbe")
relax_calc = matcalc.RelaxCalc( # type: ignore
    calculator_pbe,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True
)

data = relax_calc.calc(struct_expt)


In [ ]:
final_structure_pbe = data['final_structure']
print(final_structure_pbe)
final_structure_pbe

In [ ]:
phonon_calc = matcalc.PhononCalc( # type: ignore
    calculator_pbe,
    relax_structure=True,
    write_band_structure=f"output/{reduced_formula}/pbe_phonon_bs.yaml",
    write_total_dos=f"output/{reduced_formula}/pbe_phonon_dos.dat",
    write_phonon=f"output/{reduced_formula}/pbe_phonon.yaml",
    optimizer="FIRE",
)

data_phonon_pbe = phonon_calc.calc(final_structure_pbe)

In [ ]:
import matplotlib.pyplot as plt

phonon_bs = data_phonon_pbe['phonon'].band_structure

n_axes = sum(1 for c in phonon_bs.path_connections if not c)

fig, axs = plt.subplots(1, n_axes, figsize=(16, 5))

if n_axes == 1:
    axs = [axs]

for ax in axs[1:]:
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelleft=False)

phonon_bs.plot(ax=axs)
fig.suptitle("Phonon band structure", fontsize=16)
plt.subplots_adjust(wspace=0.07)
plt.show()


In [ ]:
phonon_dos = data_phonon_pbe["phonon"].total_dos

fig, ax = plt.subplots(figsize=(8, 5))
phonon_dos.plot(ax=ax)
fig.suptitle("Phonon DOS", fontsize=16)
plt.show()

In [ ]:
plt.plot(
    data_phonon_pbe['thermal_properties']['temperatures'],
    data_phonon_pbe['thermal_properties']['heat_capacity']
)

plt.xlabel("Temperature (K)")
plt.ylabel('Heat Capacity (J/mol-K)')
plt.title("Heat Capacity vs Temperature")
plt.show()

## r2scan

In [ ]:
## Relax with r2scan

import matcalc

calculator_r2scan = matcalc.load_fp("r2scan")
relax_calc = matcalc.RelaxCalc(
    calculator_r2scan,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True
)

data_r2scan = relax_calc.calc(struct_expt)


In [ ]:
final_structure_r2scan = data_r2scan['final_structure']
print(final_structure_r2scan)
final_structure_r2scan

In [ ]:
phonon_calc = matcalc.PhononCalc( # type: ignore
    calculator_r2scan,
    relax_structure=True,
    write_band_structure=f"output/{reduced_formula}/r2scan_phonon_bs.yaml",
    write_total_dos=f"output/{reduced_formula}/r2scan_phonon_dos.dat",
    write_phonon=f"output/{reduced_formula}/r2scan_phonon.yaml",
    optimizer="FIRE",
)

data_phonon_r2scan = phonon_calc.calc(final_structure_r2scan)

In [ ]:
import matplotlib.pyplot as plt

phonon_bs = data_phonon_r2scan['phonon'].band_structure

n_axes = sum(1 for c in phonon_bs.path_connections if not c)

fig, axs = plt.subplots(1, n_axes, figsize=(16, 5))

if n_axes == 1:
    axs = [axs]

for ax in axs[1:]:
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelleft=False)

phonon_bs.plot(ax=axs)
fig.suptitle("Phonon band structure", fontsize=16)
plt.subplots_adjust(wspace=0.07)
plt.show()


In [ ]:
phonon_dos = data_phonon_r2scan["phonon"].total_dos

fig, ax = plt.subplots(figsize=(8, 5))
phonon_dos.plot(ax=ax)
fig.suptitle("Phonon DOS", fontsize=16)
plt.show()

In [ ]:
plt.plot(
    data_phonon_pbe['thermal_properties']['temperatures'],
    data_phonon_pbe['thermal_properties']['heat_capacity']
)

plt.xlabel("Temperature (K)")
plt.ylabel('Heat Capacity (J/mol-K)')
plt.title("Heat Capacity vs Temperature")
plt.show()